In [2]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(r'../cleaned/output_encoded.csv')
df.head()

,fuel_type,mileage [km],model,name,price [pln],seller_type,year,engine_capacity [cm3],horsepower [km],is_gearbox_automatic,region
0,178162.801039,62000,101613.193302,85505.357299,63000,90852.118289,2019,1368,145,0,80689.855237
1,178162.801039,56800,101613.193302,133323.671059,99000,90852.118289,2018,1368,165,0,128266.815789
2,178162.801039,59000,101613.193302,85505.357299,71000,90852.118289,2018,1368,165,0,215862.601274
3,178162.801039,80089,101613.193302,85505.357299,59500,90852.118289,2016,1368,164,0,128266.815789
4,178162.801039,33000,101613.193302,85505.357299,70000,265742.166860,2021,1368,145,0,104437.423222


In [4]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()

y = df['price [pln]'].values
X = df.drop(columns=['price [pln]'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).ravel()

In [12]:
param_grid = {
    "C": np.arange(1, 100.0, 1.0),
    "epsilon": np.arange(0.0, 1.0, 0.1),
    "gamma": np.arange(0.0, 1.0, 0.1),
}

svr = SVR(kernel='rbf')
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train_scaled)

print(grid_search.best_estimator_)
best_svr = grid_search.best_estimator_

SVR(C=np.float64(11.0), epsilon=np.float64(0.1), gamma=np.float64(0.1))


In [13]:
svr = SVR(kernel='rbf', C=11, epsilon=0.1, gamma=0.1)

svr.fit(X_train_scaled, y_train_scaled)
y_pred_scaled = svr.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE dla SVR: {rmse}")

RMSE dla SVR: 86382.01461525857


In [14]:
svr.score(X_test_scaled, y_test_scaled)

0.9200707434473265